![](image/icon1.png) ![](image/icon2.png) ![](image/icon3.png) 

    
>**Projet Informatique L2** **UT3 Paul Sabatier**  
>**Paul Chiapello**   
>**Nathan DELPORTE**

## Projet RayTracing
***

## Objectif:
***
L'objectif de ce projet consiste à créer une image à partir de lancer de rayons.
Pour cela il a fallu créer plusieurs élements telles que la scène en y intégrant des objets comme des sphères et plans faisant office de surface et permettant une impression de profondeur.
Pour finir on a essayé de créer un petit film mettant en scène une balle qui rebondi à partir d’un enchaînement de création d’image.
***
Avant de commencer le projet, nous avons procédé à l'installation de nos interpréteurs Python et de nos terminaux à l'aide de GitBash. Nous avons ensuite créé un dépôt sur GitHub pour travailler en collaboration, voici le lien : [Projet_Info_Image](https://github.com/Paulciapello/Projet_Info_Image.git)

Nous avons décidé d'utiliser l'interpréteur Jupyter en raison de sa simplicité et de la possibilité de créer un rapport directement dans l'environnement.

## Déroulé du projet: 
***

1) Création des variables des objets sur la scène
2) Création des normales et des rayons
3) Calcul des intersections avec les objets
4) Mise en places des éclairages, des ombres et des réflections
5) Paramètre renseignés pour l’image
6) Mise en place de l'animation
7) Projet Initial (Echouée)
***

### I.	Création des dictionnaires des objets sur la scène
***
Chaque objet nous les avons rangé dans des fonctions qui faisaient en quelque sorte office de répertoire pour leur variables respectives.
En effet leur variables respective qui se définissent sous forme de liste sont les propriétés de l'objet. On peut voir que chaque propriété est identifiée grâce à une balise. Et chaque balise se définit grâce au variable appelé lors de la création de la fonction avec des “np.array”.


#### Rayon

In [2]:
def create_Ray(O, D):
    ray = {'origin': np.array(O),
           'direction': np.array(D)}
    return ray

#### Plan

In [3]:
def create_plane(P, n, amb, dif, sp,ref, i):
    plane = {'type' : 'plane',
             'position': np.array(P),
             'vect_n': np.array(n),
             'diffuse': np.array(dif),
             'ambient' : np.array(amb),
             'specular': np.array(sp),
             'reflection': ref,
             'index' : int(i)}
    return plane

#### Sphere

In [5]:
def create_sphere(A,lim,C, r, amb, dif, sp, ref, i):
    sphere = {'type':'sphere',                                                                  
              'axe':np.array(A),
              'height':np.array(lim),
              'centre': np.array(C),
              'rayon': np.array(r),
              'diffuse': np.array(dif),
              'ambient' : np.array(amb),
              'specular': np.array(sp),
              'reflection': ref,
              'index' : int(i)}
    return sphere

### II.	Création des normales et des rayons
***

#### Fonction de Normalisation

In [ ]:
def normalize(x):
    return x/np.linalg.norm(x)

#### Rayon definie pour tous t

In [ ]:
def rayAt(ray, t):
    return ray['origin']+t*ray['direction']

#### Fonction de normal aux surfaces 

In [ ]:
def get_Normal(obj, M):
    if obj['type'] == 'sphere':
        Vect =normalize(M-obj['centre'])
    elif obj['type'] == 'plane':
        Vect = obj['vect_n']
    elif obj['type'] == 'cylinder':
        H = obj['centre']
        Vect = normalize(M[0]-H[0],M[1]-H[1],0)
    
    return Vect

### IV. Calculs des intersections avec les objets


***

#### Fonction d'intersection rayon/plan

In [ ]:
def intersect_Plane(ray, plane):
    P = plane['position']
    n = plane['vect_n']
    O = ray['origin']
    d = ray['direction']
    if abs(np.dot(d, n)) < 1e-6:
        return np.inf
    t = -np.dot((O-P),n)/np.dot(d, n)
    if t>0:
        return t
    return np.inf

#### Fonction d'intersection rayon/sphere

In [1]:
def intersect_Sphere(ray, sphere):
    O = ray['origin']
    d = ray['direction']
    C = sphere['centre']
    r = sphere['rayon']

    a = np.dot(d, d)
    b = -2 * np.dot(d, (C-O))
    c = np.dot((C-O),(C-O)) - r**2

    delta = b**2 - 4*a*c
    if delta >= 0:
        t1 = (-b-np.sqrt(delta))/(2*a)
        t2 = (-b+np.sqrt(delta))/(2*a)
        if t1 >= 0 and t2 >= 0:
            return min(t1, t2)
        else:
            return np.inf
    else:
        return np.inf

#### Fonction d'intersection rayon/cylindre

In [2]:
def intersect_Cylinder(ray, cylinder):
    O = ray['origin']
    d = ray['direction']
    C = cylinder['centre']
    R = cylinder['rayon']
    z = 10
    
    x1 = O[0]
    y1 = O[1]       #récupere les (x1,y1,z1) de O
    #O_z = O[2]
    
    a = d[0]
    b = d[1]       #récupere les (a,b,c) du vecteur direction
    #c = d[2]
    
    x0 = C[0]
    y0 = C[1]       #récupere les(xo,yo,zo) de C
    z0 = C[2]
    
    A = a**2 + b**2
    B = 2 * (x1 * a - x0 * a - y1 * b + y0 * b)
    C = x1**2 - 2 * x1 * x0 + x0**2 + y1**2 - 2 * y1 * y0 + y0**2 - R**2 + (z - z0)**2
    
    delta = B**2 - 4 * A * C
    
    if delta >= 0:
        t_1 = (-B-np.sqrt(delta))/(2*A)
        t_2 = (-B+np.sqrt(delta))/(2*A)
        if t_1 >= 0 and t_2 >= 0:
            return min(t_1, t_2)
        else:
            return np.inf
    else:
        return np.inf
    
def intersect_Scene(ray, obj):
    if obj['type'] == 'plane':
        return intersect_Plane(ray, obj)
    elif obj['type'] == 'sphere':
        return intersect_Sphere(ray, obj)
    elif obj['type'] == 'cylinder':
        return intersect_Cylinder(ray, obj)

### V. Mise en places des eclairages, des ombres et des réflections

***

L'objectif de ce code est de voir si le point "P" qui est sur le plan à une normale "N" qui a une intersection avec un objet vers la source lumineuse. Si l'objet est dans l'ombre cela nous renverra un "true" sinon un false. Ce booléen sera pris en compte dans la fonction "trace_ray".

In [3]:
def is_in_shadow(obj_min, P, N):
    """Verifie si les points intersecté sont dans l'ombre"""
    PL = normalize(L - P)                          # Calcul la direction de la lumière 
    ray_test = create_ray(P + acne_eps * N, PL)    # Crée un rayon d'origine P + acne_eps * N dans la direction de la lumière        
    I_intersect = []                       # Listera tous les points intersecté

    for obj in scene:
        if obj['index'] != obj_min['index']:            
            t_obj = intersect_scene(ray_test, obj)
            if t_obj != np.inf:
                I_intersect.append(t_obj)

    if I_intersect:         
        return True              
    return False

Puis l’eclairage, on a récupéré les normal entre Le plan et la lumière mais aussi le plan et la sphère dans des variable respective (PL et PC) mais aussi la normal des objets grâce à la variable N qui prend la valeur du get_Normal. 
A partir de là, il y a “ambient” pour la couleur de l’objet. 
Le “diffuse”  va émettre la lumière, couleur dans plusieurs directions.
Le “specular” qui lui va être en quelque sorte le rond blanc serait la surface dont la normale est la plus proche de la source lumineuse.


In [5]:
def eclairage(obj,Light,P): 
    N = get_Normal(obj, P)
    PL = normalize(L-P)
    PC = normalize(C-P)
#on calcule la couleur suivant les modèles utilisé
    ct = obj['ambient']*Light['ambient']
#Lambert shading (diffuse).
    ct += obj['diffuse']*Light['diffuse']*max(np.dot(N,PL),0)
#Blinn-Phong shading (specular).
    ct += obj['specular']*Light['specular']*max(np.dot(N,normalize(PL+PC)),0)**(materialShininess)
    return ct

Pour la réflection nous avons écrit le code ci-dessous. la formule suivante permet de calculé la durection du rayon après réfelections. Avec les variables "dir_ray" et "N" pour la normale à l'objets qui va emettre son reflet.

In [2]:
def reflected_ray(dir_ray, N):
    """Calcul la direction du rayon réfléchi."""
    return dir_ray - 2 * np.dot(dir_ray, N) * N

Après la réflections il est necessaire de gérer jusqu'à où la réflexion se fait mais aussi mettre à jour la couleur de l'objet qui sera intersecté par le rayon créé par la fonction "reflected_ray". Comme on peut le voir ci-dessous, la fonction est une boucle "for" qui permet de répéter la réflexion jusqu'à que la variable "traced" soit égale à "none". On peut voir que la variable "col" ce mets à jour avec la réflexion grâce à sa formule "col = col + c * col_ray".<br>
On peut voir aussi que la variable "c" permet de gérer la distance jusqu'à laquel la réflection se fait grâce  à la formule suivante "c = c * obj['reflection']".

In [3]:
def compute_reflection(ray_test, depth_max, col):
    """Calcul la couleur de la réfelction"""
    d = ray_test['direction']
    c = 1

    for k in range(1, depth_max):
        traced = trace_ray(ray_test)

        if traced is None:
            break

        obj, M, N, col_ray = traced
        col = col + c * col_ray
        d = reflected_ray(ray_test['direction'], N)
        ray_test = create_ray(M + acne_eps * N, d)
        c = c * obj['reflection']

    return col

Pour finir la fonction "trace_ray". On peut voir que la fonction commence par une boucle "for" pour verifier la distance minimum de l'objet grâce à l'inéquation "if t_obj < t_min:" cette boucle if permet de mettre à jour la position "obj_min" et "t_min" sinon cela renvoit juste un "none" à la fonction.<br>
Dans un deuxième temps on a mis une boucle qui permet la verification de la présence d'une ombre ou non. S'il y en a la boucle "if" renvoie un "none" sinon elle calcule la couleur du rayon avec la l'eclairage de la scène.<br>
Pour finir le return renvoie toute les valeurs à la fonction "trace_ray".

In [ ]:
def trace_ray(ray):
    """Trace un rayon à travers la scene"""
    t_min = np.inf
    obj_min = None

    for obj in scene:
        t_obj = intersect_scene(ray, obj)
        if t_obj < t_min:
            t_min, obj_min = t_obj, obj

    if obj_min is None:
        return None

    P = ray_at(ray, t_min)
    N = get_normal(obj_min, P)
    shadow = is_in_shadow(obj_min, P, N)

    if shadow:
        return None

    col_ray = eclairage(obj_min, Light, P)
    return obj_min, P, N, col_ray

### VI. Paramètre renseignés pour l'image

***

Pour finir nous avons mis les paramètres d'entrée pour notre scène c’est à dire les couleurs, le niveaux des reflets, les dimensions des objets, la création des rayons, pour finir l’enregistrement de l’image, etc.
Ci-dessous quelques exemple intéressants : 

In [ ]:
# Position et couleur de la source lumineuse
Light = { 'position': np.array([5, 5, 0]),
          'ambient': np.array([0.05, 0.05, 0.05]),
          'diffuse': np.array([1, 1, 1]),
          'specular': np.array([1, 1, 1]) }

L = Light['position']

Les paramètres donnent couleurs et les positions de la lumière on peut reconnaître d’ailleurs les balises qui sont utilisées régulièrement dans les codes vue précédemment.

In [ ]:
create_sphere([x, y, z], # Position
                             .6, # Rayon
                             np.array([1. , 0.6, 0. ]), # ambiant
                             np.array([1. , 0.6, 0. ]), # diffuse
                             np.array([1, 1, 1]), # specular
                             0.2, # reflection index
                             1), # index

Exemple des données renseignées pour un objet. On peut voir qu’en texte on a indiqué les correspondances de chaque liste. En effet l’ordre est important, il doit être le même que dans la fonction créée précédemment.

In [ ]:
 # Loop through all pixels.
    for i, x in enumerate(np.linspace(S[0], S[2], w)):
        if i % 10 == 0:
            print(i / float(w) * 100, "%")
        for j, y in enumerate(np.linspace(S[1], S[3], h)):
            col = np.zeros((3))
            Q[:2] = (x,y)
            D = normalize(Q-C)
            rayTest = create_ray(C, D)
            traced = trace_ray(rayTest)
            if traced :
                obj, M, N, col_ray = traced
                col = compute_reflection(rayTest,depth_max,col)
            img[h - j - 1, i, :] = np.clip(col, 0, 1) # la fonction clip permet de "forcer" col a être dans [0,1]
    
    plt.imsave('figRaytracing'+str(u)+'.png', img)

Ci-dessus on peut voir les boucles “for” qui permettent la création de l’image c’est elle qui va finalement générer l’image et prenant en compte que les rayons dans la scène.
On peut voire juste après la fonction qui permet d'enregistrer l’image dans le fichier où est enregistré le code.

![](image/Image.png)

### VII. Mise en place du film

***

Nous avons créé une image ci-dessus, mais maintenant, pour réaliser une vidéo d'une balle qui rebondit, nous avons besoin d'un nombre conséquent d'images avec la balle se déplaçant à chaque image.

Pour cela, nous avons implémenté une boucle for de 0 à 1000, modifiant la valeur du np.array "centre" à chaque itération, afin de capturer le mouvement de la balle à travers les images.

Pour permettre le rebondissement de la balle, nous avons créé une fonction avec les constantes a, b, c définies. Nous avons ensuite récupéré ces 1000 images et créé une vidéo.

Vous pouvez visionner la vidéo en suivant ce lien : [Animation](https://github.com/Paulciapello/Projet_Info_Image.git)


In [ ]:
for u in range(0,1000):     # Calcul des 1000 images 
    
    a = -1.2
    b = -1.6
    c = 0.1
    x = u/40 - 1.5
    z = -u/40
    y = np.abs(np.exp(-c * z) * a * np.cos((2 * np.pi / b) * z))-0.3   # fonction trouvé à l'aide de matplotlib ou nous pouvons réglé l'amplitude la periode et la limite
   
    scene = [create_sphere([x, y, z], # Position
                             .6, # Rayon
                             np.array([1. , 0.6, 0. ]), # ambiant
                             np.array([1. , 0.6, 0. ]), # diffuse
                             np.array([1, 1, 1]), # specular
                             0.4, # reflection index
                             1), # index
              create_plane([0., -.9, 0.], # Position
                             [0, 1, 0], # Normal
                             np.array([0.145, 0.584, 0.854]), # ambiant
                             np.array([0.145, 0.584, 0.854]), # diffuse
                             np.array([1, 1, 1]), # specular
                             0.7, # reflection index
                             2),# index
              create_cylinder([.75, -.3, -1.], 10.,3., np.array([1. , 0.6, 0. ]),
                              np.array([1. , 0.6, 0. ]), np.array([1, 1, 1]), 0.2, 3)
             ]
    
    # Loop through all pixels.
    for i, x in enumerate(np.linspace(S[0], S[2], w)):
        if i % 10 == 0:
            print(i / float(w) * 100, "%")
        for j, y in enumerate(np.linspace(S[1], S[3], h)):
            col = np.zeros((3))
            Q[:2] = (x,y)
            D = normalize(Q-C)
            rayTest = create_ray(C, D)
            traced = trace_ray(rayTest)
            if traced :
                obj, M, N, col_ray = traced
                col = compute_reflection(rayTest,depth_max,col)
            img[h - j - 1, i, :] = np.clip(col, 0, 1) # la fonction clip permet de "forcer" col a être dans [0,1]
    
    plt.imsave('figRaytracing'+str(u)+'.png', img)
    u = u + 1

### VIII Projet Initial (Echoué)

Notre projet initial était de créer une cage de foot à l'aide de cylindres et de réaliser une animation avec la balle entrant dans la cage, faisant office de but. Malheureusement, cela n'a pas marché, nous n'avons pas réussi à implémenter les cylindres